## Modify Data accrording previous conclusions

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import sys
sys.path.insert(0, 'G:/work/GitHub/ml_baseline_things/functions/')
import supportFunctions as sf
import featureEngineringFunctions as fef

dirPath = '2.1-featuresPack/'


## Conclusions from previous notebook:

* drop features:
 * passengerId
 * cabin (after extract the deck)
 * ticket
 * embarked
* missing data to recover:
 * <s>age</s>
* convert categorial to:
 * <s>sex - to int and to several columns</s>
* new features from:
 * <s>cabin (extract decks)</s>
 * <s>name (extract title)</s>
 * <s>age bands</s>
 * <s>fare </s>normalize and (?)<s> make bands</s>
 * <s>sibsp & parch - may be generated isAlone hasChildren/Parents hasHasb/Wife/</s>
 
 Finally convert to smaller memory size

In [2]:
dataRawTrain, dataRawTest, _ = sf.load_raw_data()
dataTrain, dataTest, _ = sf.load_raw_data()
#dataTrain.head()

Train (891, 12); Test (418, 11); Total (1309, 12).
Train (891, 12); Test (418, 11); Total (1309, 12).


## Id + Survived

In [3]:
# save base attrs
fef.save_to_file(dataTrain['PassengerId'], dataTest['PassengerId'], dirPath, 'PassengerId')
featureName = 'Survived'
_ = joblib.dump(dataTrain[featureName], open('{}{}_train.pkl'.format(dirPath, featureName), 'wb'), 9)

PassengerId saved.


### Class

In [4]:
fef.save_to_file (dataTrain['Pclass'], dataTest['Pclass'], dirPath, 'Pclass_cnt')

#save as OHE
featureTrain, col_ = fef.fe_cat_ohe(dataTrain, pd.concat([dataTrain, dataTest]), 'Pclass')
featureTest, col_ = fef.fe_cat_ohe(dataTest, pd.concat([dataTrain, dataTest]), 'Pclass')
fef.save_to_file (featureTrain, featureTest, dirPath, 'Pclass_OHE') 

Pclass_cnt saved.
Pclass_OHE saved.


0

### Missing data recovery, SEX

In [5]:
#no missing values

# encode SEX feature

#save as OHE
featureTrain, col_ = fef.fe_cat_ohe(dataTrain, pd.concat([dataTrain, dataTest]), 'Sex')
featureTest, col_ = fef.fe_cat_ohe(dataTest, pd.concat([dataTrain, dataTest]), 'Sex')
fef.save_to_file (featureTrain, featureTest, dirPath, 'SEX_OHE')

#save as LE
featureTrain, col_ = fef.fe_cat_le(dataTrain, pd.concat([dataTrain, dataTest]), 'Sex')
featureTest, col_ = fef.fe_cat_le(dataTest, pd.concat([dataTrain, dataTest]), 'Sex')
fef.save_to_file (featureTrain, featureTest, dirPath, 'SEX_LE')


for dataset in [dataTrain, dataTest]:
    #dataset['Sex'] = dataset['Sex'].map({'female': 1, 'male': 0}).astype(int)
    feature, _ = fef.fe_cat_le(dataset, pd.concat([dataTrain, dataTest]), 'Sex')
    dataset['Sex_LE'] = feature
    
for dataset in [dataTrain, dataTest]:
    dataset.drop(['Sex'], axis=1, inplace=True)

SEX_OHE saved.
SEX_LE saved.


In [6]:
dataTrain.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_LE
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,1


### Missing data recovery, AGE
Strategy = median Age for Pclass=1 and Gender=0, Pclass=1 and Gender=1, and so on...

In [7]:
sf.check_missing_data(dataTrain, dataTest, ['Age'])
#dataTrain.loc[dataTrain['Age'].isnull() == True]

Train:
 Age    177
dtype: int64 

Test:
 Age    86
dtype: int64


0

In [8]:
guess_ages = np.zeros((2,3))

for dataset in [dataTrain, dataTest]:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex_LE'] == i) & (dataset['Pclass'] == j+1)]['Age'].dropna()
            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int(age_guess/0.5 + 0.5) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex_LE == i) & (dataset.Pclass == j+1), 'Age'] = guess_ages[i,j]

#dataTrain.head()
sf.check_missing_data(dataTrain, dataTest, ['Age'])

Train:
 Age    0
dtype: int64 

Test:
 Age    0
dtype: int64


0

In [9]:
fef.save_to_file (dataTrain['Age'], dataTest['Age'], dirPath, 'AGE_cnt')

AGE_cnt saved.


0

### Extract title from name

In [10]:
for dataset in [dataTrain, dataTest]:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    
dataTrain.Title.value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Col           2
Major         2
Lady          1
Capt          1
Countess      1
Sir           1
Mme           1
Don           1
Ms            1
Jonkheer      1
Name: Title, dtype: int64

In [11]:
for dataset in [dataTrain, dataTest]:
    dataset['Title'] = dataset['Title'].replace(['Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace(['Mlle','Ms', 'Lady'], 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
dataTrain[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.704301
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.318182


In [12]:
#old way
#title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

#for dataset in [dataTrain, dataTest]:
#    dataset['Title'] = dataset['Title'].map(title_mapping)

#current way
for dataset in [dataTrain, dataTest]:
    feature, _ = fef.fe_cat_le(dataset, pd.concat([dataTrain, dataTest]), 'Title')
    dataset['Title_LE'] = feature

#save as OHE
featureTrain, col_ = fef.fe_cat_ohe(dataTrain, pd.concat([dataTrain, dataTest]), 'Title')
featureTest, col_ = fef.fe_cat_ohe(dataTest, pd.concat([dataTrain, dataTest]), 'Title')
fef.save_to_file (featureTrain, featureTest, dirPath, 'Title_OHE')    

#save as LE
featureTrain, col_ = fef.fe_cat_le(dataTrain, pd.concat([dataTrain, dataTest]), 'Title')
featureTest, col_ = fef.fe_cat_le(dataTest, pd.concat([dataTrain, dataTest]), 'Title')
fef.save_to_file (featureTrain, featureTest, dirPath, 'Title_LE')

for dataset in [dataTrain, dataTest]:
    dataset.drop(['Title'], axis=1, inplace=True)
    
sf.check_unique_data(dataTrain, dataTest, 'Title_LE')
#sf.check_missing_data(dataTrain, dataTest, ['Title'])



Title_OHE saved.
Title_LE saved.
Train unique values:
 [2 3 1 0 4] 

Test unique values:
 [2 3 1 0 4]


0

### Extract AgeBands from Age

In [13]:
dataTrain['AgeBand'] = pd.cut(dataTrain['Age'], 5)
dataTrain[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 16.336]",0.550000
1,"(16.336, 32.252]",0.336714
2,"(32.252, 48.168]",0.412844
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80.0]",0.090909


In [14]:
#all data
pd.cut(pd.concat([dataTrain, dataTest])['Age'], 5).value_counts().sort_index(ascending=True)
#dataTrain[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

(0.0902, 16.136]    134
(16.136, 32.102]    748
(32.102, 48.068]    308
(48.068, 64.034]    106
(64.034, 80.0]       13
Name: Age, dtype: int64

In [15]:
for dataset in [dataTrain, dataTest]:
    dataset['Age_lev'] = 0
    dataset['Age_lev'].loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32)] = 1
    dataset['Age_lev'].loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48)] = 2
    dataset['Age_lev'].loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64)] = 3
    dataset['Age_lev'].loc[(dataset['Age'] > 64)] = 4

#save
fef.save_to_file (dataTrain['Age_lev'], dataTest['Age_lev'], dirPath, 'Age_lev')  

sf.check_unique_data(dataTrain, dataTest, 'Age_lev')

dataTrain.drop('AgeBand', axis=1, inplace=True)

Age_lev saved.
Train unique values:
 [1 2 3 0 4] 

Test unique values:
 [2 3 1 0 4]


In [16]:
dataTest.head()

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_LE,Title_LE,Age_lev
0,892,3,"Kelly, Mr. James",34.5,0,0,330911,7.8292,NaN,Q,1,2,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",47.0,1,0,363272,7.0000,NaN,S,0,3,2
2,894,2,"Myles, Mr. Thomas Francis",62.0,0,0,240276,9.6875,NaN,Q,1,2,3
3,895,3,"Wirz, Mr. Albert",27.0,0,0,315154,8.6625,NaN,S,1,2,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",22.0,1,1,3101298,12.2875,NaN,S,0,3,1


### sibsp & parch new features generation

In [17]:
for dataset in [dataTrain, dataTest]:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

fef.save_to_file (dataTrain['FamilySize'], dataTest['FamilySize'], dirPath, 'FamilySize_cnt')
    
dataTrain[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

FamilySize_cnt saved.


,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [18]:
for dataset in [dataTrain, dataTest]:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    dataset['HasWifeHasb'] = 0
    dataset.loc[dataset['SibSp'] > 0 , 'HasWifeHasb'] = 1
    dataset['HasBloodRelatives'] = 0
    dataset.loc[dataset['Parch'] > 0 , 'HasWifeHasb'] = 1
    
fef.save_to_file (dataTrain['IsAlone'], dataTest['IsAlone'], dirPath, 'IsAlone')
fef.save_to_file (dataTrain['HasWifeHasb'], dataTest['HasWifeHasb'], dirPath, 'HasWifeHasb')
fef.save_to_file (dataTrain['HasBloodRelatives'], dataTest['HasBloodRelatives'], dirPath, 'HasBloodRelatives')

dataTrain[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

IsAlone saved.
HasWifeHasb saved.
HasBloodRelatives saved.


,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [19]:
dataTrain

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_LE,Title_LE,Age_lev,FamilySize,IsAlone,HasWifeHasb,HasBloodRelatives
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,1,2,1,2,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,0,3,2,2,0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,1,1,1,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,0,3,2,2,0,1,0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,1,2,2,1,1,0,0
5,6,0,3,"Moran, Mr. James",25.0,0,0,330877,8.4583,NaN,Q,1,2,1,1,1,0,0
6,7,0,1,"McCarthy, Mr. Timothy J",54.0,0,0,17463,51.8625,E46,S,1,2,3,1,1,0,0
7,8,0,3,"Palsson, Master. Gosta Leonard",2.0,3,1,349909,21.0750,NaN,S,1,0,0,5,0,1,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",27.0,0,2,347742,11.1333,NaN,S,0,3,1,3,0,1,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",14.0,1,0,237736,30.0708,NaN,C,0,3,0,2,0,1,0


### Syntetical union of features

In [20]:
for dataset in [dataTrain, dataTest]:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

fef.save_to_file (dataTrain['Age*Class'], dataTest['Age*Class'], dirPath, 'Age-Class_cnt')
    
dataTrain.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

Age-Class_cnt saved.


,Age*Class,Age,Pclass
0,66.0,22.0,3
1,38.0,38.0,1
2,78.0,26.0,3
3,35.0,35.0,1
4,105.0,35.0,3
5,75.0,25.0,3
6,54.0,54.0,1
7,6.0,2.0,3
8,81.0,27.0,3
9,28.0,14.0,2


### Fare levels

In [21]:
sf.check_missing_data(dataTrain, dataTest, ['Fare'])

Train:
 Fare    0
dtype: int64 

Test:
 Fare    1
dtype: int64


0

In [22]:
#for test only
def get_median_fare_by_item (pClass, i):
    return dataTest.query('Pclass == {}'.format(pClass))['Fare'].mean()
    
for i in range(len (dataTest)):
    if pd.isnull(dataTest.loc[i,'Fare']):
        dataTest.loc[i,'Fare'] = get_median_fare_by_item (dataTest.loc[i,'Pclass'], i)

In [23]:
#dataTest.query('Pclass == 3')['Fare'].mean()
#dataTest.loc[dataTest['Fare'].isnull() == True]
#dataTest.loc[152,'Fare']#=None
#dataTest.loc[dataTest['Fare'].isnull() == True]

In [24]:
dataTrain['FareBand'] = pd.qcut(dataTrain['Fare'], 5)
dataTrain[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.854]",0.217877
1,"(7.854, 10.5]",0.201087
2,"(10.5, 21.679]",0.424419
3,"(21.679, 39.688]",0.444444
4,"(39.688, 512.329]",0.642045


In [25]:
for dataset in [dataTrain, dataTest]:
    dataset['Fare_lev'] = 0
    dataset['Fare_lev'].loc[dataset['Fare'] <= 7.85] = 0
    dataset['Fare_lev'].loc[(dataset['Fare'] > 7.85) & (dataset['Fare'] <= 10.5)] = 1
    dataset['Fare_lev'].loc[(dataset['Fare'] > 10.5) & (dataset['Fare'] <= 21.67)] = 2
    dataset['Fare_lev'].loc[(dataset['Fare'] > 21.67) & (dataset['Fare'] <= 39.7)] = 3
    dataset['Fare_lev'].loc[dataset['Fare'] > 39.7] = 4

fef.save_to_file (dataTrain['Fare_lev'], dataTest['Fare_lev'], dirPath, 'Fare_lev') 

dataTrain = dataTrain.drop(['FareBand'], axis=1)

fef.save_to_file (dataTrain['Fare'], dataTest['Fare'], dirPath, 'Fare_cnt')

for dataset in [dataTrain, dataTest]:
    dataset.drop(['Fare'], axis=1)

Fare_lev saved.
Fare_cnt saved.


### Cabin

In [26]:
#decks = {"U": 0, "A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "T": 0}

for dataset in [dataTrain, dataTest]:
    dataset['Deck'] = 0
    #dataset['Deck'] = dataset['Cabin'].fillna('U').apply(lambda c: decks.get(c[0], -1))
    dataset['Deck'] = dataset['Cabin'].fillna('U').apply(lambda c: c[0])

In [27]:
print (dataTrain.Deck.value_counts())
print (dataTest.Deck.value_counts())

for dataset in [dataTrain, dataTest]:
    dataset['Deck'] = dataset['Deck'].replace('T', 'U')

print (dataTrain.Deck.value_counts())
print (dataTest.Deck.value_counts())

U    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: Deck, dtype: int64
U    327
C     35
B     18
D     13
E      9
F      8
A      7
G      1
Name: Deck, dtype: int64
U    688
C     59
B     47
D     33
E     32
A     15
F     13
G      4
Name: Deck, dtype: int64
U    327
C     35
B     18
D     13
E      9
F      8
A      7
G      1
Name: Deck, dtype: int64


In [28]:
#save as OHE
featureTrain, col_ = fef.fe_cat_ohe(dataTrain, pd.concat([dataTrain, dataTest]), 'Deck')
featureTest, col_ = fef.fe_cat_ohe(dataTest, pd.concat([dataTrain, dataTest]), 'Deck')
fef.save_to_file (featureTrain, featureTest, dirPath, 'Deck_OHE')    

#save as LE
featureTrain, col_ = fef.fe_cat_le(dataTrain, pd.concat([dataTrain, dataTest]), 'Deck')
featureTest, col_ = fef.fe_cat_le(dataTest, pd.concat([dataTrain, dataTest]), 'Deck')
fef.save_to_file (featureTrain, featureTest, dirPath, 'Deck_LE')

for dataset in [dataTrain, dataTest]:
    feature, _ = fef.fe_cat_le(dataset, pd.concat([dataTrain, dataTest]), 'Deck')
    dataset['Deck_LE'] = feature

for dataset in [dataTrain, dataTest]:
    feature, _ = fef.fe_cat_le(dataset, pd.concat([dataTrain, dataTest]), 'Deck')
    dataset['Deck_LE'] = feature

for dataset in [dataTrain, dataTest]:
    dataset.drop(['Deck'], axis=1, inplace=True)   
    dataset.drop(['Cabin'], axis=1, inplace=True) 

#for dataset in [dataTrain, dataTest]:
#    deckDummies = pd.get_dummies(dataset['Cabin'].fillna('U').apply(lambda c: c[0])
#                              .replace('T', 'U'), prefix='Deck', dummy_na=False)
#    dataset = pd.concat([dataset, deckDummies], axis=1)
    
#dataTrain.head()

sf.check_unique_data(dataTrain, dataTest, 'Deck_LE')

Deck_OHE saved.
Deck_LE saved.
Train unique values:
 [7 2 4 6 3 0 1 5] 

Test unique values:
 [7 1 4 0 2 3 5 6]


0

### Drop features

In [29]:
dataTrain.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Embarked,Sex_LE,Title_LE,Age_lev,FamilySize,IsAlone,HasWifeHasb,HasBloodRelatives,Age*Class,Fare_lev,Deck_LE
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,S,1,2,1,2,0,1,0,66.0,0,7
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C,0,3,2,2,0,1,0,38.0,4,2
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,S,0,1,1,1,1,0,0,78.0,1,7
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,S,0,3,2,2,0,1,0,35.0,4,2
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,S,1,2,2,1,1,0,0,105.0,1,7


In [30]:
#* drop features:
# * passengerId
# * cabin (after extract the deck)
# * ticket
# * embarked
#to be dropped
toBeDropped = ['PassengerId', 'Ticket', 'Embarked', 'Name']

dataTrainTarget = dataTrain["Survived"]

dataTrain = dataTrain.drop(toBeDropped, axis=1)
dataTrain = dataTrain.drop('Survived', axis=1)

dataTest = dataTest.drop(toBeDropped, axis=1)

dataTrain.shape, dataTrainTarget.shape, dataTest.shape

((891, 15), (891,), (418, 15))

In [31]:
dataTrain.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_LE,Title_LE,Age_lev,FamilySize,IsAlone,HasWifeHasb,HasBloodRelatives,Age*Class,Fare_lev,Deck_LE
0,3,22.0,1,0,7.2500,1,2,1,2,0,1,0,66.0,0,7
1,1,38.0,1,0,71.2833,0,3,2,2,0,1,0,38.0,4,2
2,3,26.0,0,0,7.9250,0,1,1,1,1,0,0,78.0,1,7
3,1,35.0,1,0,53.1000,0,3,2,2,0,1,0,35.0,4,2
4,3,35.0,0,0,8.0500,1,2,2,1,1,0,0,105.0,1,7


In [32]:
print (dataRawTrain.columns)
dataRawTrain.head()
#PassengerId +
#Survived +
#Pclass +
#Name +
#Sex +
#Age +
#SibSp + 
#Parch +
#Ticket +
#Fare +
#Cabin +
#Embarked +

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Final checkers missing values

In [33]:
sf.check_missing_data(dataTrain, dataTest)
#TODO неплохо было бы проверять на уникальные значения в тесте и трейне - чтобы бились

Train:
 Pclass               0
Age                  0
SibSp                0
Parch                0
Fare                 0
Sex_LE               0
Title_LE             0
Age_lev              0
FamilySize           0
IsAlone              0
HasWifeHasb          0
HasBloodRelatives    0
Age*Class            0
Fare_lev             0
Deck_LE              0
dtype: int64 

Test:
 Pclass               0
Age                  0
SibSp                0
Parch                0
Fare                 0
Sex_LE               0
Title_LE             0
Age_lev              0
FamilySize           0
IsAlone              0
HasWifeHasb          0
HasBloodRelatives    0
Age*Class            0
Fare_lev             0
Deck_LE              0
dtype: int64


0

### Save data to dump

In [34]:
#TODO дату в файле
#@comment_if_you_are_ready
#_ = joblib.dump([dataTrain, dataTrainTarget, dataTest, dataTrainScaled, dataTestScaled], 
#            open('data_modified/dataModified.pkl', 'wb'), 9)

## what can be done more

In [35]:
- можно восстановить палубу исходя из класса и ценника билета
1 класс - A-E, самые дорогие в B
2 и 3 класс - D-G, в D - лучшие

- при разбиении возраста разбивал по cut, а не по qcut
- Syntetical union of features

SyntaxError: invalid syntax (<ipython-input-35-177e344abc58>, line 1)